# Loading the dataset

In [1]:
from deeplake.core.vectorstore.deeplake_vectorstore import VectorStore
import os

In [2]:
os.environ['ACTIVELOOP_TOKEN'] = "eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpZCI6ImhyYXlyIiwiYXBpX2tleSI6IkFzUXFwaDNmWFBqLTRpbkZiM0IyWmRWU1VmRmNWQWtwQThwZkg4cVFRX29fWiJ9."

In [3]:
import deeplake

In [4]:
deeplake_path = 'hub://hrayr/audio_dataset3'

In [5]:
try:
    ds = deeplake.empty(deeplake_path)
except:
    ds = deeplake.load(deeplake_path)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/hrayr/audio_dataset3



/

hub://hrayr/audio_dataset3 loaded successfully.



In [6]:
try:
    with ds:
        ds.create_tensor('audio', htype = 'audio', sample_compression = None, exist_ok=True)
        ds.create_tensor('metadata', htype = 'json', exist_ok=True)
except ValueError:
    pass

In [7]:
ds.summary()

Dataset(path='hub://hrayr/audio_dataset3', tensors=['audio', 'metadata'])

  tensor    htype      shape       dtype  compression
 -------   -------    -------     -------  ------- 
  audio     audio   (3, 1323000)  float64   None   
 metadata   json       (3, 1)       str     None   


# Populating the dataset

In [8]:
from glob import glob
import random
import librosa
from pathlib import Path
import json

In [11]:
all_music_files = glob('../Dataset/raw_music/train/*.wav')

In [12]:
indices_to_take = random.sample(range(len(all_music_files)), 20)

In [13]:
metadata = []
metadata_file = Path('./egs/train/data.jsonl')

with open(str(metadata_file), "r") as filled_json_file:
    for index, line in enumerate(filled_json_file):
        if index in indices_to_take:
            link_info_dict = json.loads(line)
            metadata.append(link_info_dict)

In [ ]:
for i, data in enumerate(metadata):
    print(i)
    music_path = Path(data['path'])
    json_path = music_path.with_suffix('.json')
    audio, sr = librosa.load(music_path, sr=None, mono=True)

    with open(json_path, 'r') as json_file:
        json_info = json.load(json_file)

    full_meta = {'metadata': data, 'info': json_info}

    ds.append({'audio': audio, 'metadata': full_meta})
    

0


-

In [17]:
ds.summary()

Dataset(path='hub://hrayr/audio_dataset3', tensors=['audio', 'metadata'])

  tensor    htype          shape           dtype  compression
 -------   -------        -------         -------  ------- 
  audio     audio   (24, 749700:1323000)  float64   None   
 metadata   json          (24, 1)           str     None   
